# <B> Training </B>
* Container: codna_pytorch_p39

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 1. parameter store 설정

In [2]:
import boto3
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
prefix = pm.get_params(key="PREFIX")

## 2.Training-job for preprocessing

In [5]:
import os
import sagemaker
from omegaconf import OmegaConf
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.processing import ProcessingInput, ProcessingOutput, FrameworkProcessor

* **Set Up SageMaker Experiment**
    - Create or load [SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) for the example training job. This will create an experiment trial object in SageMaker.

In [6]:
from time import strftime
from smexperiments.trial import Trial
from smexperiments.experiment import Experiment

In [7]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name)

In [8]:
def create_trial(experiment_name):
    create_date = strftime("%m%d-%H%M%s")
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

* **Configure the training job**

    - Now we configure the training job, by modifying the `config.yaml` file that is stored in our source code directory.
    - We pass relative directory paths for the data based on the SageMaker mount directory on the remote instance.

In [9]:
code_dir = "./code"
config_dir = "./code/conf"
config_path = os.path.join(config_dir, "config.yaml")

* params for training job

In [10]:
# Set to True to enable SageMaker to run locally
local_mode = True


if local_mode:
    instance_type = "local_gpu"
    
    from sagemaker.local import LocalSession
    import os
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    
    local_data_dir = os.getcwd() + '/data/preprocessing'
    pretrained = os.getcwd() + '/pretrained/CTC.nemo'
    
    data_channels = {"training": f"file://{local_data_dir}", "testing": f"file://{local_data_dir}", "pretrained": f"file://{pretrained}"}
    
else:
    
    instance_type = "ml.p3.2xlarge" #"ml.p3.2xlarge"#"ml.g4dn.8xlarge"#"ml.p3.2xlarge", 'ml.p3.16xlarge' , ml.g4dn.8xlarge
    
    sagemaker_session = sagemaker.Session()
    data_channels = {"training": pm.get_params(key="-".join([prefix, "PREP-DATA-PATH"])), 
                     "testing": pm.get_params(key="-".join([prefix, "PREP-DATA-PATH"])), 
                     "pretrained": pm.get_params(key=prefix + "-PRETRAINED-WEIGHT")}

instance_count = 1
do_spot_training = False
max_wait = None
max_run = 1*60*60   

resume = True
    
proc_prefix = "/opt/ml/processing"
bucket_name = pm.get_params(key=prefix + "-BUCKET")

output_path = os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "training",
    "model-output"
)

code_location = os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "training",
    "backup_codes"
)

experiment_name = '-'.join([prefix, "nemo-experiments"])

num_re = "([0-9\\.]+)(e-?[[01][0-9])?"
metric_definitions = [
    {"Name": "train_loss", "Regex": f"loss={num_re}"},
    {"Name": "wer", "Regex": f"wer:{num_re}"}
]

kwargs = {}

In [11]:
print("experiment_name : {} \ntrain_instance_type : {} \ntrain_instance_count : {}\ndata_channels : {}\nexperiment_name : {}".format(experiment_name, instance_type, instance_count, data_channels, experiment_name))    

experiment_name : nemo-asr-nemo-experiments 
train_instance_type : local_gpu 
train_instance_count : 1
data_channels : {'training': 'file:///home/ec2-user/SageMaker/nemo-on-sagemaker/1.building-component/data/preprocessing', 'testing': 'file:///home/ec2-user/SageMaker/nemo-on-sagemaker/1.building-component/data/preprocessing', 'pretrained': 'file:///home/ec2-user/SageMaker/nemo-on-sagemaker/1.building-component/pretrained/CTC.nemo'}
experiment_name : nemo-asr-nemo-experiments


* config

In [12]:
conf = OmegaConf.load(config_path)

# Sampling
conf.model.sample_rate = 16000

# Set Data Locations based on the mounted directory in the SageMaker instance
conf.model.train_ds.manifest_filepath = "/opt/ml/input/data/training/an4/train_manifest.json"
conf.model.validation_ds.manifest_filepath = "/opt/ml/input/data/testing/an4/test_manifest.json"
# training setup
conf.trainer.accelerator = "gpu"
conf.trainer.num_nodes = instance_count
# enable SageMaker DDP
conf.trainer.strategy = None #"ddp"
conf.trainer.max_epochs = 2

# Output directory for our experiment within the SageMaker instance
conf.exp_manager.exp_dir="/opt/ml/model/"

# Create a Small Variant of the Conformer Model
conf.model.encoder.n_layers = 8
conf.model.n_heads = 4
conf.model.spec_augment.time_masks = 5

# Set Optimizer parameters
conf.model.optim.lr = 2.0 # by default we using Noam scheduling, the LR is a multiplier

if resume == False:
    # resume flags if crashes occur
    conf.exp_manager.resume_if_exists=False 
    conf.exp_manager.resume_ignore_no_checkpoint=False
    conf.init_from_nemo_model = None
    
else:
    # resume flags if crashes occur
    conf.exp_manager.resume_if_exists=True
    conf.exp_manager.resume_ignore_no_checkpoint=True
    # the pre-trained model we want to fine-tune
    conf.init_from_nemo_model = "/opt/ml/input/data/pretrained/CTC.nemo"
    

OmegaConf.save(conf, config_path)

* Define processing job

In [13]:
pm.get_params(key=''.join([prefix, "-IMAGE-URI"])), code_dir

('419974056037.dkr.ecr.us-east-1.amazonaws.com/nemo-test-training', './code')

In [14]:
est = PyTorch(
    entry_point="speech_to_text_ctc.py", # the script we want to run
    source_dir=code_dir, # where our conf/script is
    role=pm.get_params(key=prefix + "-SAGEMAKER-ROLE-ARN"),
    instance_type=instance_type,
    instance_count=instance_count,
    image_uri=pm.get_params(key=''.join([prefix, "-IMAGE-URI"])),
    # framework_version="1.13.1", # version of PyTorch
    # py_version="py39",
    volume_size=512,
    code_location = code_location,
    output_path=output_path,
    disable_profiler=True,
    debugger_hook_config=False,
    hyperparameters={'config-path': 'conf'},
    #distribution={"smdistributed":{"dataparallel":{"enabled":True, "fp16": True}}},
    sagemaker_session=sagemaker_session,
    metric_definitions=metric_definitions,
    max_run=max_run,
    enable_sagemaker_metrics=True,
    **kwargs
)

* run

In [15]:
if instance_type =='local_gpu': est.checkpoint_s3_uri = None

create_experiment(experiment_name)
job_name = create_trial(experiment_name)

est.fit(
    inputs=data_channels, 
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=True,
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating training-job with name: nemo-asr-nemo-experiments-0322-05441679463868
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-bht9p:
    command: train
    container_name: 4dnwh7ma7e-algo-1-bht9p
    deploy:
      resources:
        reservations:
          

Creating 4dnwh7ma7e-algo-1-bht9p ... 
Creating 4dnwh7ma7e-algo-1-bht9p ... done
Attaching to 4dnwh7ma7e-algo-1-bht9p
4dnwh7ma7e-algo-1-bht9p | 2023-03-22 05:44:31,758 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
4dnwh7ma7e-algo-1-bht9p | 2023-03-22 05:44:31,782 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
4dnwh7ma7e-algo-1-bht9p | 2023-03-22 05:44:31,789 sagemaker-training-toolkit INFO     instance_groups entry not present in resource_config
4dnwh7ma7e-algo-1-bht9p | 2023-03-22 05:44:31,792 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
4dnwh7ma7e-algo-1-bht9p | 2023-03-22 05:44:31,797 sagemaker_pytorch_container.training INFO     Invoking user training script.
4dnwh7ma7e-algo-1-bht9p | 2023-03-22 05:44:31,822 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
4dnwh7ma7e-algo-1-bht9p | 2023-03-22 05:44:31,829 sagemaker

* save model-path, experiment-name

In [152]:
pm.put_params(key="-".join([prefix, "MODEL-PATH"]), value=est.model_data, overwrite=True)
pm.put_params(key="-".join([prefix, "EXPERI-NAME"]), value=experiment_name, overwrite=True)

'Store suceess'

In [153]:
job_name

'nemo-asr-nemo-experiments-0321-04351679373354'

* show experiments

In [132]:
from sagemaker.analytics import ExperimentAnalytics
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100

In [133]:
trial_component_training_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    #sort_by="metrics.validation:auc.max",        
    #sort_order="Descending",
    #metric_names=["validation:auc"]
)

trial_component_training_analytics.dataframe()[['Experiments', 'Trials', \
                                                'wer - Last', 'wer - Min', 'wer - Max', 'wer - Avg', \
                                                'train_loss - Last', 'train_loss - Min', 'train_loss - Max', 'train_loss - Avg']]

AttributeError: 'LocalSagemakerClient' object has no attribute 'search'

In [ ]:
from sagemaker.analytics import TrainingJobAnalytics
df = TrainingJobAnalytics( training_job_name=job_name).dataframe()

In [53]:
df

,timestamp,metric_name,value
0,0.0,train_loss,76.866667
1,0.0,wer,1170.000000


## code pipeline trigger

In [134]:
import boto3
pipeline_client = boto3.client('codepipeline')

In [ ]:
import boto3
pipeline_client = boto3.client('codepipeline')
code_pipeline_name = "mlops-code-pipeline"
pipeline_client.start_pipeline_execution(name=code_pipeline_name)